In [ ]:
# 📊 方法1: GitHub経由でデータセットを取得
# プロジェクトがGitHubにプッシュされている場合

USE_GITHUB = False  # ← Trueに変更してGitHubリポジトリURLを設定
GITHUB_REPO_URL = ""  # ← あなたのGitHubリポジトリURLを入力

if USE_GITHUB:
    import os
    
    # GitHubからプロジェクトをクローン
    repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
    
    if not os.path.exists(repo_name):
        !git clone {GITHUB_REPO_URL}
        print(f"✅ Repository cloned: {repo_name}")
    else:
        print(f"📁 Repository already exists: {repo_name}")
    
    # データセットファイルの確認
    dataset_path = f"{repo_name}/data/dpo_dataset.jsonl"
    
    if os.path.exists(dataset_path):
        # データセット情報を表示
        with open(dataset_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        print(f"📊 Dataset found: {len(lines)} samples")
        print(f"📁 Dataset path: {dataset_path}")
        
        # 最初の数行を表示
        import json
        print("\n🔍 First sample:")
        sample = json.loads(lines[0])
        for key, value in sample.items():
            if isinstance(value, str) and len(value) > 100:
                print(f"{key}: {value[:100]}...")
            else:
                print(f"{key}: {value}")
    else:
        print(f"❌ Dataset not found at {dataset_path}")
        print("Please check your repository structure or use another method.")
else:
    print("📝 GitHub method not enabled. Set USE_GITHUB = True if needed.")

In [ ]:
# 📊 方法2: Google Driveにアップロードして使用
# データセットファイルをGoogle Driveにアップロードしてから使用

USE_GOOGLE_DRIVE = False  # ← Trueに変更してGoogle Driveを使用

if USE_GOOGLE_DRIVE:
    from google.colab import drive, files
    import os
    import json
    
    # Google Driveをマウント
    drive.mount('/content/drive')
    print("✅ Google Drive mounted")
    
    # オプション1: ファイルをアップロード
    print("\n📤 Option 1: Upload dataset file directly")
    print("Run the next cell to upload your dpo_dataset.jsonl file")
    
    # オプション2: Google Driveから読み込み
    print("\n📁 Option 2: Place file in Google Drive")
    print("1. Upload dpo_dataset.jsonl to your Google Drive")
    print("2. Update the path below to match your file location")
    
    # Google Driveのパスを設定（ユーザーが調整）
    drive_dataset_path = "/content/drive/MyDrive/dpo_dataset.jsonl"  # ← パスを調整
    
    if os.path.exists(drive_dataset_path):
        with open(drive_dataset_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        print(f"📊 Dataset found: {len(lines)} samples")
        
        # データセットを作業ディレクトリにコピー
        !cp "{drive_dataset_path}" ./dpo_dataset.jsonl
        print("✅ Dataset copied to working directory")
        
        # サンプル表示
        sample = json.loads(lines[0])
        print("\n🔍 First sample:")
        for key, value in sample.items():
            if isinstance(value, str) and len(value) > 100:
                print(f"{key}: {value[:100]}...")
            else:
                print(f"{key}: {value}")
    else:
        print(f"❌ Dataset not found at {drive_dataset_path}")
        print("Please check the file path or upload the file first.")
else:
    print("📝 Google Drive method not enabled. Set USE_GOOGLE_DRIVE = True if needed.")

In [ ]:
# 📤 Google Colabにファイルを直接アップロード
# Google Driveを使わずに直接ファイルをアップロード

UPLOAD_FILE = False  # ← Trueに変更してファイルをアップロード

if UPLOAD_FILE:
    from google.colab import files
    import json
    import os
    
    print("📤 Please select your dpo_dataset.jsonl file to upload...")
    uploaded = files.upload()
    
    # アップロードされたファイルを確認
    for filename in uploaded.keys():
        print(f"✅ Uploaded: {filename} ({len(uploaded[filename])} bytes)")
        
        if filename.endswith('.jsonl'):
            # データセット情報を表示
            with open(filename, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            print(f"📊 Dataset loaded: {len(lines)} samples")
            
            # ファイル名を標準化
            if filename != 'dpo_dataset.jsonl':
                os.rename(filename, 'dpo_dataset.jsonl')
                print("✅ File renamed to dpo_dataset.jsonl")
            
            # サンプル表示
            sample = json.loads(lines[0])
            print("\n🔍 First sample:")
            for key, value in sample.items():
                if isinstance(value, str) and len(value) > 100:
                    print(f"{key}: {value[:100]}...")
                else:
                    print(f"{key}: {value}")
            break
else:
    print("📝 File upload not enabled. Set UPLOAD_FILE = True if needed.")

In [ ]:
# 📊 方法3: データセットを直接作成（小さなサンプル用）
# 少数のサンプルでテストする場合

CREATE_SAMPLE_DATASET = False  # ← Trueに変更してサンプルデータセットを作成

if CREATE_SAMPLE_DATASET:
    import json
    
    # サンプルDPOデータセット（広告技術分野）
    sample_dataset = [
        {
            "prompt": "プログラマティック広告の主要なメリットは何ですか？",
            "chosen": "プログラマティック広告の主要なメリットには、リアルタイムでの入札最適化、精密なターゲティング、効率的な広告費の活用、そして大規模なインベントリへのアクセスが含まれます。これにより、広告主はより関連性の高いオーディエンスにリーチし、ROIを最大化できます。",
            "rejected": "プログラマティック広告は自動化された広告配信システムです。便利で効率的です。"
        },
        {
            "prompt": "RTB（Real-Time Bidding）の仕組みを説明してください。",
            "chosen": "RTBは、ユーザーがウェブページを訪問した瞬間にリアルタイムで広告枠のオークションが行われる仕組みです。DSP（Demand Side Platform）が自動的に入札を行い、最高価格を提示した広告主の広告が表示されます。この過程は通常100ミリ秒以内で完了し、ユーザー体験を損なうことなく最適な広告マッチングを実現します。",
            "rejected": "RTBは広告をリアルタイムで買う方法です。オークション形式で行われます。"
        },
        {
            "prompt": "DMPとCDPの違いは何ですか？",
            "chosen": "DMP（Data Management Platform）は主に匿名化されたサードパーティデータを管理し、オーディエンスセグメンテーションに特化したプラットフォームです。一方、CDP（Customer Data Platform）はファーストパーティデータを中心とした顧客データの統合・管理を行い、個人レベルでの顧客体験最適化を目的としています。CDPはより包括的な顧客データ管理とプライバシー重視の設計が特徴です。",
            "rejected": "DMPとCDPはどちらもデータを管理するプラットフォームです。機能は似ています。"
        }
    ]
    
    # データセットファイルを作成
    with open('dpo_dataset.jsonl', 'w', encoding='utf-8') as f:
        for item in sample_dataset:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"✅ Sample dataset created: {len(sample_dataset)} samples")
    print("📁 File saved as: dpo_dataset.jsonl")
    
    # サンプル表示
    print("\n🔍 Sample data:")
    for i, item in enumerate(sample_dataset[:2], 1):
        print(f"\nSample {i}:")
        print(f"Prompt: {item['prompt'][:80]}...")
        print(f"Chosen: {item['chosen'][:80]}...")
        print(f"Rejected: {item['rejected'][:80]}...")
else:
    print("📝 Sample dataset creation not enabled. Set CREATE_SAMPLE_DATASET = True if needed.")

In [ ]:
# 🔍 データセット検証とトレーニング準備
# どの方法でもデータセットが準備できたら、このセルで検証とトレーニング準備を行います

import os
import json
from pathlib import Path

# データセットファイルの存在確認
dataset_file = 'dpo_dataset.jsonl'

if os.path.exists(dataset_file):
    print(f"✅ Dataset file found: {dataset_file}")
    
    # データセットの詳細分析
    with open(dataset_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    print(f"📊 Total samples: {len(lines)}")
    
    # データ形式の検証
    valid_samples = 0
    invalid_samples = 0
    sample_lengths = {'prompt': [], 'chosen': [], 'rejected': []}
    
    for i, line in enumerate(lines[:10]):  # 最初の10サンプルを検証
        try:
            data = json.loads(line.strip())
            if 'prompt' in data and 'chosen' in data and 'rejected' in data:
                valid_samples += 1
                sample_lengths['prompt'].append(len(data['prompt']))
                sample_lengths['chosen'].append(len(data['chosen']))
                sample_lengths['rejected'].append(len(data['rejected']))
            else:
                invalid_samples += 1
                print(f"⚠️ Sample {i+1} missing required fields")
        except json.JSONDecodeError:
            invalid_samples += 1
            print(f"❌ Sample {i+1} invalid JSON format")
    
    print(f"✅ Valid samples (first 10): {valid_samples}")
    print(f"❌ Invalid samples (first 10): {invalid_samples}")
    
    if valid_samples > 0:
        # 平均文字数を計算
        avg_lengths = {
            'prompt': sum(sample_lengths['prompt']) / len(sample_lengths['prompt']),
            'chosen': sum(sample_lengths['chosen']) / len(sample_lengths['chosen']),
            'rejected': sum(sample_lengths['rejected']) / len(sample_lengths['rejected'])
        }
        
        print("\n📏 Average text lengths:")
        for field, avg_len in avg_lengths.items():
            print(f"  {field}: {avg_len:.1f} characters")
        
        # サンプル表示
        print("\n🔍 Sample data preview:")
        sample = json.loads(lines[0])
        print(f"Prompt: {sample['prompt'][:100]}...")
        print(f"Chosen: {sample['chosen'][:100]}...")
        print(f"Rejected: {sample['rejected'][:100]}...")
        
        print("\n🎯 Dataset is ready for DPO training!")
else:
    print(f"❌ Dataset file not found: {dataset_file}")
    print("Please run one of the dataset preparation methods above first.")

In [ ]:
# ⚙️ DPOトレーニングパラメータの設定
# データセットサイズに応じてトレーニングパラメータを調整

import os

if os.path.exists('dpo_dataset.jsonl'):
    with open('dpo_dataset.jsonl', 'r', encoding='utf-8') as f:
        num_samples = len(f.readlines())
    
    print(f"📊 Dataset size: {num_samples} samples")
    
    # データセットサイズに応じたパラメータ調整
    if num_samples >= 3000:
        # 大規模データセット用パラメータ
        training_config = {
            'num_train_epochs': 3,
            'per_device_train_batch_size': 2,
            'gradient_accumulation_steps': 8,
            'learning_rate': 5e-7,
            'warmup_steps': 100,
            'max_steps': 1000,
            'eval_steps': 100,
            'save_steps': 200,
            'max_length': 512,
            'max_prompt_length': 256
        }
        print("🎯 Using parameters optimized for large dataset (3000+ samples)")
    elif num_samples >= 1000:
        # 中規模データセット用パラメータ
        training_config = {
            'num_train_epochs': 5,
            'per_device_train_batch_size': 4,
            'gradient_accumulation_steps': 4,
            'learning_rate': 1e-6,
            'warmup_steps': 50,
            'max_steps': 500,
            'eval_steps': 50,
            'save_steps': 100,
            'max_length': 512,
            'max_prompt_length': 256
        }
        print("🎯 Using parameters optimized for medium dataset (1000-3000 samples)")
    else:
        # 小規模データセット用パラメータ
        training_config = {
            'num_train_epochs': 10,
            'per_device_train_batch_size': 8,
            'gradient_accumulation_steps': 2,
            'learning_rate': 2e-6,
            'warmup_steps': 20,
            'max_steps': 200,
            'eval_steps': 20,
            'save_steps': 50,
            'max_length': 512,
            'max_prompt_length': 256
        }
        print("🎯 Using parameters optimized for small dataset (<1000 samples)")
    
    print("\n📋 Training Configuration:")
    for key, value in training_config.items():
        print(f"  {key}: {value}")
    
    # 推定トレーニング時間
    estimated_time_minutes = (training_config['max_steps'] * training_config['per_device_train_batch_size'] * 0.5) / 60
    print(f"\n⏰ Estimated training time: {estimated_time_minutes:.1f} minutes")
    
    # GPU使用量の警告
    if training_config['per_device_train_batch_size'] >= 4:
        print("\n⚠️  Warning: Large batch size may cause GPU memory issues.")
        print("   If you encounter CUDA out of memory errors, reduce batch_size to 1-2.")
else:
    print("❌ Please prepare the dataset first using one of the methods above.")

In [ ]:
# 🤖 DPOトレーニングの実行
# 実際のDPOトレーニングを開始します

RUN_DPO_TRAINING = False  # ← Trueに変更してトレーニングを開始

if RUN_DPO_TRAINING:
    import torch
    from transformers import (
        AutoModelForCausalLM,
        AutoTokenizer,
        TrainingArguments
    )
    from trl import DPOTrainer
    from datasets import Dataset
    import json
    import os
    
    # GPU確認
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🔧 Using device: {device}")
    
    if torch.cuda.is_available():
        print(f"   GPU: {torch.cuda.get_device_name(0)}")
        print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    # ベースモデルの選択
    model_options = {
        "1": {
            "name": "microsoft/DialoGPT-small",
            "description": "Small conversational model (117M params) - Fast training"
        },
        "2": {
            "name": "rinna/japanese-gpt2-small",
            "description": "Japanese GPT-2 Small (110M params) - Japanese optimized"
        },
        "3": {
            "name": "elyza/ELYZA-japanese-Llama-2-7b-fast-instruct",
            "description": "Japanese Llama-2 7B - High quality but requires more GPU memory"
        }
    }
    
    print("\n🤖 Available base models:")
    for key, model in model_options.items():
        print(f"  {key}. {model['name']}")
        print(f"     {model['description']}")
    
    # デフォルトでGPT-2 Smallを使用（GPU メモリ効率的）
    selected_model = model_options["2"]["name"]
    print(f"\n✅ Selected model: {selected_model}")
    
    try:
        # トークナイザーとモデルの読み込み
        print("📥 Loading tokenizer and model...")
        tokenizer = AutoTokenizer.from_pretrained(selected_model)
        
        # パディングトークンの設定
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        model = AutoModelForCausalLM.from_pretrained(
            selected_model,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
        
        print("✅ Model and tokenizer loaded successfully")
        
        # データセットの読み込み
        print("📊 Loading dataset...")
        dataset_list = []
        
        with open('dpo_dataset.jsonl', 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line.strip())
                dataset_list.append({
                    'prompt': data['prompt'],
                    'chosen': data['chosen'],
                    'rejected': data['rejected']
                })
        
        # データセットを作成
        dataset = Dataset.from_list(dataset_list)
        
        # 訓練・検証データの分割
        split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
        train_dataset = split_dataset['train']
        eval_dataset = split_dataset['test']
        
        print(f"✅ Dataset loaded: {len(train_dataset)} train, {len(eval_dataset)} eval samples")
        
    except Exception as e:
        print(f"❌ Error loading model or dataset: {str(e)}")
        print("Please check your model selection and dataset file.")
        RUN_DPO_TRAINING = False
else:
    print("📝 DPO training not enabled. Set RUN_DPO_TRAINING = True to start training.")
    print("\n⚠️  Before starting training:")
    print("   1. Ensure your dataset is prepared and validated")
    print("   2. Check GPU memory availability")
    print("   3. Review training parameters above")

In [ ]:
# 🚀 DPOトレーニング実行の続き
# 前のセルでモデルとデータセットが正常にロードされた場合にのみ実行

if RUN_DPO_TRAINING and 'model' in locals() and 'train_dataset' in locals():
    try:
        # トレーニング引数の設定
        print("⚙️ Setting up training arguments...")
        
        training_args = TrainingArguments(
            output_dir="./dpo_output",
            num_train_epochs=training_config['num_train_epochs'],
            per_device_train_batch_size=training_config['per_device_train_batch_size'],
            per_device_eval_batch_size=training_config['per_device_train_batch_size'],
            gradient_accumulation_steps=training_config['gradient_accumulation_steps'],
            learning_rate=training_config['learning_rate'],
            warmup_steps=training_config['warmup_steps'],
            max_steps=training_config['max_steps'],
            eval_steps=training_config['eval_steps'],
            save_steps=training_config['save_steps'],
            logging_steps=20,
            evaluation_strategy="steps",
            save_strategy="steps",
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            report_to=None,  # Wandbなどのロギングを無効化
            remove_unused_columns=False,
            dataloader_pin_memory=False,
            fp16=torch.cuda.is_available(),  # GPU使用時のみfp16を有効化
        )
        
        # DPOトレーナーの初期化
        print("🎯 Initializing DPO trainer...")
        
        dpo_trainer = DPOTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=tokenizer,
            max_length=training_config['max_length'],
            max_prompt_length=training_config['max_prompt_length'],
            beta=0.1,  # DPO温度パラメータ
        )
        
        print("✅ DPO trainer initialized successfully")
        
        # トレーニング開始
        print("\n🚀 Starting DPO training...")
        print(f"   Training samples: {len(train_dataset)}")
        print(f"   Validation samples: {len(eval_dataset)}")
        print(f"   Max steps: {training_config['max_steps']}")
        print(f"   Estimated time: {estimated_time_minutes:.1f} minutes")
        print("\n   Training will start now. You can monitor progress below.")
        
        # 実際のトレーニング実行
        dpo_trainer.train()
        
        print("\n🎉 DPO training completed successfully!")
        
        # モデルの保存
        print("💾 Saving the trained model...")
        dpo_trainer.save_model("./dpo_final_model")
        tokenizer.save_pretrained("./dpo_final_model")
        
        print("✅ Model saved to: ./dpo_final_model")
        
        # 訓練統計
        if hasattr(dpo_trainer.state, 'log_history'):
            logs = dpo_trainer.state.log_history
            if logs:
                final_train_loss = None
                final_eval_loss = None
                
                for log in reversed(logs):
                    if 'train_loss' in log and final_train_loss is None:
                        final_train_loss = log['train_loss']
                    if 'eval_loss' in log and final_eval_loss is None:
                        final_eval_loss = log['eval_loss']
                    if final_train_loss is not None and final_eval_loss is not None:
                        break
                
                print("\n📊 Training Results:")
                if final_train_loss:
                    print(f"   Final training loss: {final_train_loss:.4f}")
                if final_eval_loss:
                    print(f"   Final validation loss: {final_eval_loss:.4f}")
        
    except Exception as e:
        print(f"❌ Training failed: {str(e)}")
        print("\nPossible solutions:")
        print("   1. Reduce batch_size in training configuration")
        print("   2. Reduce max_length or max_prompt_length")
        print("   3. Try a smaller model")
        print("   4. Restart runtime and try again")
elif RUN_DPO_TRAINING:
    print("❌ Cannot start training: Model or dataset not properly loaded")
    print("Please run the previous cells first.")
else:
    print("📝 Training not started. Set RUN_DPO_TRAINING = True in the previous cell.")

In [ ]:
# 🧪 学習済みモデルのテスト
# DPOトレーニング完了後、モデルの性能をテストします

TEST_TRAINED_MODEL = False  # ← Trueに変更してモデルをテスト

if TEST_TRAINED_MODEL:
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer
    import os
    
    model_path = "./dpo_final_model"
    
    if os.path.exists(model_path):
        print("🤖 Loading trained DPO model...")
        
        try:
            # 学習済みモデルとトークナイザーをロード
            tokenizer = AutoTokenizer.from_pretrained(model_path)
            model = AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None
            )
            
            print("✅ Model loaded successfully")
            
            # テスト用プロンプト（広告技術分野）
            test_prompts = [
                "プログラマティック広告の主要なメリットは何ですか？",
                "RTB（Real-Time Bidding）の仕組みを説明してください。",
                "DMPとCDPの違いは何ですか？",
                "広告配信における機械学習の活用方法を教えてください。",
                "Cookie廃止が広告業界に与える影響について説明してください。"
            ]
            
            print("\n🧪 Testing the trained model:")
            print("=" * 60)
            
            for i, prompt in enumerate(test_prompts, 1):
                print(f"\n🔍 Test {i}: {prompt}")
                print("-" * 50)
                
                # テキスト生成
                inputs = tokenizer.encode(prompt, return_tensors="pt")
                
                with torch.no_grad():
                    outputs = model.generate(
                        inputs,
                        max_length=inputs.shape[1] + 150,
                        num_return_sequences=1,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=tokenizer.eos_token_id,
                        no_repeat_ngram_size=2
                    )
                
                generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                response = generated_text[len(prompt):].strip()
                
                print(f"📝 Response: {response}")
                
                if i < len(test_prompts):  # 最後でない場合
                    print()
            
            print("\n" + "=" * 60)
            print("🎉 Model testing completed!")
            print("\n💡 Tips for evaluation:")
            print("   • Compare responses with original model")
            print("   • Check for relevant technical terminology")
            print("   • Assess factual accuracy and coherence")
            print("   • Look for improved domain-specific knowledge")
            
        except Exception as e:
            print(f"❌ Error loading or testing model: {str(e)}")
    
    else:
        print(f"❌ Trained model not found at {model_path}")
        print("Please complete the DPO training first.")
else:
    print("📝 Model testing not enabled. Set TEST_TRAINED_MODEL = True to test the model.")
    print("\n📋 To test the model:")
    print("   1. Complete DPO training first")
    print("   2. Set TEST_TRAINED_MODEL = True")
    print("   3. Run this cell to see model responses")

In [ ]:
# 💾 プロジェクトの保存と次のステップ
# トレーニング完了後のファイル管理と次のステップガイド

SAVE_PROJECT = False  # ← Trueに変更してプロジェクトファイルを保存

if SAVE_PROJECT:
    from google.colab import files
    import zipfile
    import os
    
    print("💾 Preparing project files for download...")
    
    # 保存するファイルのリスト
    files_to_save = []
    
    # データセットファイル
    if os.path.exists('dpo_dataset.jsonl'):
        files_to_save.append('dpo_dataset.jsonl')
    
    # 学習済みモデル
    if os.path.exists('./dpo_final_model'):
        model_files = []
        for root, dirs, files_list in os.walk('./dpo_final_model'):
            for file in files_list:
                file_path = os.path.join(root, file)
                model_files.append(file_path)
        files_to_save.extend(model_files)
    
    # トレーニングログ
    if os.path.exists('./dpo_output'):
        for root, dirs, files_list in os.walk('./dpo_output'):
            for file in files_list:
                if file.endswith(('.json', '.log', '.txt')):
                    file_path = os.path.join(root, file)
                    files_to_save.append(file_path)
    
    if files_to_save:
        # ZIPファイルを作成
        zip_filename = 'dpo_training_project.zip'
        
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file_path in files_to_save:
                if os.path.exists(file_path):
                    # ZIPファイル内での相対パスを保持
                    arcname = file_path.replace('./', '')
                    zipf.write(file_path, arcname)
                    print(f"  ✅ Added: {file_path}")
        
        print(f"\n📦 Created archive: {zip_filename}")
        print(f"   Total files: {len(files_to_save)}")
        
        # ファイルダウンロード
        print("\n⬇️ Downloading project archive...")
        files.download(zip_filename)
        
        print("✅ Download started! Check your Downloads folder.")
    else:
        print("❌ No project files found to save.")
        print("Please complete the training process first.")
else:
    print("📝 Project saving not enabled. Set SAVE_PROJECT = True to download files.")

# 次のステップガイド
print("\n" + "=" * 60)
print("🎯 次のステップ")
print("=" * 60)

print("\n📋 完了したタスク:")
if os.path.exists('dpo_dataset.jsonl'):
    print("   ✅ データセットの準備")
else:
    print("   ❌ データセットの準備")

if os.path.exists('./dpo_final_model'):
    print("   ✅ DPOトレーニングの実行")
else:
    print("   ❌ DPOトレーニングの実行")

if 'dpo_trainer' in locals():
    print("   ✅ モデルのテスト")
else:
    print("   ❌ モデルのテスト")

print("\n🚀 推奨される次のアクション:")
print("\n1. 📊 モデル評価の改善:")
print("   • より多様なテストプロンプトでの評価")
print("   • ベースラインモデルとの比較")
print("   • 人間による品質評価")

print("\n2. 🔧 モデルの最適化:")
print("   • ハイパーパラメータのチューニング")
print("   • より大きなデータセットでの再トレーニング")
print("   • 異なるベースモデルでの実験")

print("\n3. 🌐 本番環境への展開:")
print("   • モデルサービングAPIの構築")
print("   • 推論パフォーマンスの最適化")
print("   • セキュリティとプライバシーの考慮")

print("\n4. 📈 継続的改善:")
print("   • ユーザーフィードバックの収集")
print("   • 追加データセットでのファインチューニング")
print("   • モデル品質の監視システム構築")

print("\n" + "=" * 60)
print("🎉 Google ColabでのDPOトレーニングプロジェクト完了!")
print("=" * 60)

## 🚨 依存関係エラーが出た場合のクイックフィックス

Google Colabで以下のようなエラーが出た場合：
- `fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1`
- `gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0`

**解決手順：**
1. **Runtime → Restart runtime** をクリック
2. 下記のクイックフィックスセルを実行
3. 再度メインのセットアップセルを実行

In [ ]:
# 🚨 緊急クイックフィックス (依存関係エラー専用)
# エラーが出た場合のみ実行してください

QUICK_FIX = False  # ← Trueに変更して実行

if QUICK_FIX:
    print("🚨 Running emergency dependency fix...")
    
    # 問題のあるパッケージを強制削除
    !pip uninstall -y fastai torch gcsfs fsspec --quiet
    
    # 適切なバージョンで再インストール
    !pip install 'torch>=2.0,<2.7' --index-url https://download.pytorch.org/whl/cu118
    !pip install fsspec==2025.3.2 gcsfs==2025.3.2
    !pip install 'fastai>=2.7.0'
    
    print("✅ Quick fix completed. Now run the main setup cell below.")
else:
    print("📝 Quick fix not enabled. Set QUICK_FIX = True if needed.")

# 📊 データセットをGoogle Colabで使用する方法

既存の高品質なDPOデータセット（3,566サンプル）をGoogle Colabで使用するための3つの方法を提供します：

## 方法1: GitHub経由でデータセットを取得
## 方法2: Google Driveにアップロードして使用
## 方法3: 直接データセットをコピー&ペースト

以下のセルから適切な方法を選択してください。

# 🚀 DPO Training on Google Colab - 直接選好最適化

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/dpo-colab)

このノートブックでは、Google ColabでDPO（Direct Preference Optimization）トレーニングを実行します。

## ✨ 特徴
- 🔥 GPU加速トレーニング
- 📊 リアルタイム進捗監視
- 💾 自動チェックポイント保存
- 🎯 効率的なLoRA微調整
- 📈 結果の可視化

## ⚙️ 推奨環境
- Google Colab Pro (GPU: T4/V100)
- ランタイムタイプ: GPU

In [ ]:
# 🔧 環境セットアップ（依存関係の競合を回避）
import sys
import os

# Colab環境チェック
IN_COLAB = 'google.colab' in sys.modules
print(f"🏃‍♂️ Running in Google Colab: {IN_COLAB}")

if IN_COLAB:
    print("📦 Installing required packages with dependency resolution...")
    
    # 事前に問題のあるパッケージをアンインストール
    print("Step 1: Cleaning existing conflicting packages...")
    !pip uninstall -y fastai gcsfs fsspec torch -q
    
    print("Step 2: Installing core dependencies with specific versions...")
    !pip install -q --upgrade pip setuptools wheel
    
    # PyTorchを互換性のあるバージョンで固定
    print("Step 3: Installing PyTorch with version constraint...")
    !pip install 'torch<2.7,>=2.0' torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    
    # fsspecとgcsfsを最新バージョンで統一
    print("Step 4: Installing file system packages...")
    !pip install -q fsspec==2025.3.2 gcsfs==2025.3.2
    
    print("Step 5: Installing transformers ecosystem...")
    !pip install -q transformers datasets accelerate tokenizers
    
    print("Step 6: Installing TRL and PEFT...")
    !pip install -q trl peft
    
    print("Step 7: Installing additional ML packages...")
    !pip install -q evaluate bitsandbytes matplotlib pandas numpy
    
    # fastaiの再インストール（適切なバージョンで）
    print("Step 8: Reinstalling fastai with correct torch version...")
    !pip install -q 'fastai>=2.7.0'
    
    print("✅ Installation completed with dependency resolution!")
    
    # 警告とエラーを最小化
    import warnings
    warnings.filterwarnings('ignore')
    
    # 最終的な依存関係確認
    print("\n🔍 Verifying critical package versions:")
    import pkg_resources
    critical_packages = ['torch', 'transformers', 'trl', 'peft', 'fastai']
    for pkg in critical_packages:
        try:
            version = pkg_resources.get_distribution(pkg).version
            print(f"  ✅ {pkg}: {version}")
        except:
            print(f"  ❌ {pkg}: Not properly installed")

# GPU確認
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\n🚀 GPU Available: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    print(f"🔥 PyTorch CUDA Version: {torch.version.cuda}")
    torch.cuda.empty_cache()
else:
    print("\n⚠️ No GPU available. Please enable GPU in Runtime > Change runtime type")
    print("💡 Go to Runtime > Change runtime type > Hardware accelerator > GPU")

In [ ]:
# 🔍 トラブルシューティング: 詳細な依存関係確認

if IN_COLAB:
    print("🔍 Comprehensive package version check...")
    
    import subprocess
    import pkg_resources
    
    # 重要なパッケージの詳細確認
    packages_to_check = {
        'torch': 'PyTorch (Should be <2.7, >=2.0)',
        'transformers': 'Hugging Face Transformers',
        'trl': 'Transformer Reinforcement Learning',
        'peft': 'Parameter Efficient Fine-tuning',
        'datasets': 'Hugging Face Datasets',
        'accelerate': 'Hugging Face Accelerate',
        'fsspec': 'File System Specification (Should be 2025.3.2)',
        'gcsfs': 'Google Cloud Storage File System (Should be 2025.3.2)',
        'fastai': 'FastAI (Should work with current torch)'
    }
    
    print("\n📊 Package Status:")
    conflicts_found = False
    
    for package, description in packages_to_check.items():
        try:
            version = pkg_resources.get_distribution(package).version
            status = "✅"
            
            # 特定の競合をチェック
            if package == 'torch' and version >= '2.7':
                status = "⚠️ "
                conflicts_found = True
            elif package in ['fsspec', 'gcsfs'] and not version.startswith('2025.3'):
                status = "⚠️ "
                conflicts_found = True
                
            print(f"  {status} {package}: {version} - {description}")
        except pkg_resources.DistributionNotFound:
            print(f"  ❌ {package}: Not installed - {description}")
            conflicts_found = True
    
    if conflicts_found:
        print("\n🔧 Dependency conflicts detected! To fix:")
        print("   1. Runtime → Restart runtime")
        print("   2. Re-run the installation cell above")
        print("   3. If issues persist, enable ALTERNATIVE_INSTALL in the next cell")
    else:
        print("\n🎉 All dependencies are compatible!")

    # システム情報
    print("\n🖥️ System Information:")
    print(f"  Python: {sys.version.split()[0]}")
    print(f"  CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  GPU Count: {torch.cuda.device_count()}")

In [ ]:
# 🔄 代替インストール手法 (依存関係エラーが出た場合)

# 上記のインストールで fastai や fsspec エラーが出た場合はこちらを実行
# ⚠️ このセルを実行する前に「ランタイム → ランタイムを再起動」してください

ALTERNATIVE_INSTALL = False  # ← Trueに変更して実行

if ALTERNATIVE_INSTALL and IN_COLAB:
    print("🔄 Running alternative installation with strict version management...")
    
    # 完全クリーンアップ
    print("Step 1: Complete package cleanup...")
    !pip uninstall -y fastai fastbook torch torchvision torchaudio gcsfs fsspec transformers trl peft -q
    
    # 基盤パッケージから順番にインストール
    print("Step 2: Installing PyTorch with fastai compatibility...")
    !pip install 'torch>=1.10,<2.7' 'torchvision' 'torchaudio' --index-url https://download.pytorch.org/whl/cu118
    
    print("Step 3: Installing file system packages with exact versions...")
    !pip install fsspec==2025.3.2 gcsfs==2025.3.2
    
    print("Step 4: Installing fastai first (to avoid conflicts)...")
    !pip install 'fastai>=2.7.0,<2.8'
    
    print("Step 5: Installing transformers ecosystem...")
    !pip install transformers datasets accelerate tokenizers
    
    print("Step 6: Installing DPO training packages...")
    !pip install trl peft evaluate
    
    print("Step 7: Installing additional utilities...")
    !pip install matplotlib pandas numpy bitsandbytes
    
    print("\n✅ Alternative installation completed!")
    print("📝 Verifying installation...")
    
    # インストール確認
    try:
        import torch
        import transformers
        import trl
        import peft
        import fastai
        print("🎉 All critical packages imported successfully!")
        print(f"   PyTorch: {torch.__version__}")
        print(f"   Transformers: {transformers.__version__}")
        print(f"   FastAI: {fastai.__version__}")
    except ImportError as e:
        print(f"❌ Import error: {e}")
        print("💡 Please restart runtime and try again")

elif ALTERNATIVE_INSTALL:
    print("📝 This alternative method is only for Google Colab.")
else:
    print("📝 Alternative installation not enabled.")
    print("     Set ALTERNATIVE_INSTALL = True if you encounter dependency conflicts.")
    print("     Remember to restart runtime first!")

In [ ]:
# 📊 DPOデータセット生成
import json
import pandas as pd
import random
from typing import List, Dict

def create_dpo_dataset(size: int = 1000) -> List[Dict]:
    """高品質なDPOデータセットを生成"""
    
    templates = [
        {
            "prompt": "以下の質問に答えてください: {question}",
            "chosen": "{topic}について説明します。{detail}これにより、{benefit}が期待できます。",
            "rejected": "その質問は複雑ですね。様々な要因があります。"
        },
        {
            "prompt": "次の文章を要約してください: {text}",
            "chosen": "この文章のポイントは{point}です。具体的には{detail}について述べています。",
            "rejected": "文章が長くて複雑なため、要約は困難です。"
        },
        {
            "prompt": "プログラミングについて説明してください: {topic}",
            "chosen": "{topic}は{definition}です。主な特徴として{feature}があり、{usage}に使用されます。",
            "rejected": "プログラミングは技術的で説明が難しいです。"
        }
    ]
    
    topics = ["機械学習", "ウェブ開発", "データ分析", "人工知能", "クラウド"]
    questions = ["AIとは何ですか？", "プログラミングの基本は？", "データサイエンスとは？"]
    
    dataset = []
    for i in range(size):
        template = random.choice(templates)
        topic = random.choice(topics)
        question = random.choice(questions)
        
        if "{question}" in template["prompt"]:
            prompt = template["prompt"].format(question=question)
            chosen = template["chosen"].format(
                topic=topic,
                detail=f"{topic}の重要な概念",
                benefit="効率的な問題解決"
            )
        elif "{text}" in template["prompt"]:
            prompt = template["prompt"].format(text=f"{topic}に関する詳細な説明")
            chosen = template["chosen"].format(
                point=f"{topic}の活用",
                detail="実用的な応用例"
            )
        else:
            prompt = template["prompt"].format(topic=topic)
            chosen = template["chosen"].format(
                topic=topic,
                definition="重要な技術分野",
                feature="高い効率性",
                usage="様々な業界"
            )
        
        dataset.append({
            "prompt": prompt,
            "chosen": chosen,
            "rejected": template["rejected"]
        })
    
    return dataset

# データセット生成
print("🔄 Generating DPO dataset...")
data = create_dpo_dataset(1000)
df = pd.DataFrame(data)

print(f"✅ Generated {len(df)} samples")
print(f"📊 Columns: {list(df.columns)}")
print("\n🔍 Sample data:")
display(df.head(2))

In [ ]:
# 🤖 モデルとトークナイザー設定
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

# Colab GPU環境に最適化された軽量モデル
model_name = "microsoft/DialoGPT-small"

print(f"📥 Loading model: {model_name}")

# トークナイザー読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# モデル読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

print("✅ Model and tokenizer loaded")
print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# ⚡ LoRA設定（効率的な微調整）
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # Low rank for efficiency
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],
    bias="none"
)

# LoRAモデル作成
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

if torch.cuda.is_available():
    print(f"🔥 GPU memory usage: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

In [ ]:
# 📚 データ前処理
from datasets import Dataset
from trl import DPOTrainer, DPOConfig

# Dataset変換
dataset = Dataset.from_pandas(df)
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print(f"🎯 Training samples: {len(train_dataset)}")
print(f"🎯 Evaluation samples: {len(eval_dataset)}")

# サンプル確認
print("\n📝 Sample training data:")
print(train_dataset[0])

In [ ]:
# 🚀 DPOトレーニング設定と実行
training_args = DPOConfig(
    output_dir="./dpo_results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    fp16=True,
    gradient_checkpointing=True,
    max_length=128,
    max_prompt_length=64,
    report_to=["none"]  # Disable wandb
)

# DPOトレーナー初期化
dpo_trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    beta=0.1
)

print("⚙️ DPO Trainer initialized")
print("🔥 Starting training... This may take 10-15 minutes")

# トレーニング実行
train_result = dpo_trainer.train()

print("\n🎉 Training completed!")
print(f"📊 Final training loss: {train_result.training_loss:.4f}")

In [ ]:
# 📈 結果の評価と可視化
import matplotlib.pyplot as plt
import numpy as np

# 最終評価
eval_results = dpo_trainer.evaluate()
print("📊 Final Evaluation Results:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")

# トレーニング履歴の可視化
if hasattr(dpo_trainer.state, 'log_history'):
    log_history = dpo_trainer.state.log_history
    
    train_losses = []
    eval_losses = []
    steps = []
    
    for log in log_history:
        if 'loss' in log:
            train_losses.append(log['loss'])
            steps.append(log.get('step', len(train_losses)))
        if 'eval_loss' in log:
            eval_losses.append(log['eval_loss'])
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(steps, train_losses, 'b-', linewidth=2, label='Training Loss')
    plt.title('Training Loss Progress', fontsize=14)
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    if eval_losses:
        plt.subplot(1, 2, 2)
        eval_steps = np.linspace(0, max(steps), len(eval_losses))
        plt.plot(eval_steps, eval_losses, 'r-', linewidth=2, label='Evaluation Loss')
        plt.title('Evaluation Loss Progress', fontsize=14)
        plt.xlabel('Steps')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 🧪 トレーニング済みモデルのテスト
def generate_response(prompt, max_length=100):
    """DPOトレーニング済みモデルでレスポンス生成"""
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # プロンプト部分を除去
    if response.startswith(prompt):
        response = response[len(prompt):].strip()
    
    return response

# テスト実行
test_prompts = [
    "以下の質問に答えてください: 機械学習とは何ですか？",
    "次の文章を要約してください: プログラミングに関する詳細な説明",
    "プログラミングについて説明してください: Python"
]

print("🔬 Testing DPO-trained model...")
print("=" * 60)

for i, prompt in enumerate(test_prompts, 1):
    response = generate_response(prompt)
    print(f"\n📝 Test {i}:")
    print(f"Prompt: {prompt}")
    print(f"Response: {response}")
    print("-" * 40)

In [ ]:
# 💾 モデル保存とダウンロード
print("💾 Saving trained model...")

# モデル保存
dpo_trainer.save_model("./final_dpo_model")
tokenizer.save_pretrained("./final_dpo_model")

# ファイルサイズ確認
import os
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

model_size = get_folder_size('./final_dpo_model')
print(f"📊 Model size: {model_size / 1e6:.1f} MB")

# Google Driveに保存（オプション）
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import shutil
    drive_path = '/content/drive/MyDrive/dpo_trained_model'
    shutil.copytree('./final_dpo_model', drive_path, dirs_exist_ok=True)
    print(f"☁️ Model saved to Google Drive: {drive_path}")

# ZIP形式でダウンロード準備
!zip -r dpo_model.zip ./final_dpo_model

print("\n🎉 DPO Training Successfully Completed!")
print("\n📋 Summary:")
print(f"   • Model: {model_name}")
print(f"   • Training samples: {len(train_dataset)}")
print(f"   • Evaluation samples: {len(eval_dataset)}")
print(f"   • Final training loss: {train_result.training_loss:.4f}")
print(f"   • Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

if torch.cuda.is_available():
    print(f"   • Peak GPU memory: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

print("\n📥 To download the model:")
print("   1. Download 'dpo_model.zip' from the file browser")
print("   2. Or access from Google Drive if mounted")